In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import DataReader
from pandas import Series, DataFrame

In [2]:
# 予想前日
filedate = '20220427'

In [3]:
# 予想前日のIDNR4データ
df_idnr = pd.read_csv('data/idnr4_' + filedate + '.csv', index_col=0, header=[0], parse_dates=[0])

In [4]:
# 予想前日のタートルスープ買いデータ
df_tsplusb = pd.read_csv('data/tsplus_min_' + filedate + '.csv', index_col=0, header=[0], parse_dates=[0])

In [5]:
# 予想前日のタートルスープ売りデータ
df_tspluss = pd.read_csv('data/tsplus_max_' + filedate + '.csv', index_col=0, header=[0], parse_dates=[0])

In [6]:
# 予想前日のピンボール買いデータ
df_pinb = pd.read_csv('data/pin_80_20_買い_' + filedate + '.csv', index_col=0, header=[0], parse_dates=[0])

In [7]:
# 予想前日のピンボール売りデータ
df_pins = pd.read_csv('data/pin_80_20_売り_' + filedate + '.csv', index_col=0, header=[0], parse_dates=[0])

In [8]:
# 予想前日の株価データ
df_stock = pd.read_csv('data/stock_' + filedate + '.csv', index_col=0, header=[0,1], parse_dates=[0])

In [9]:
# indexの順番を入れ替えて、ソート
df_stock1 = df_stock.swaplevel('Attributes','Symbols', axis=1)
df_stock1.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

In [10]:
# 直近10日分のデータを取得
df_stock2 = df_stock1.tail(10).T

In [11]:
# インデックスを株価コードに、カラムをそれぞれの価格に変更
df_stock2 = df_stock2.pivot_table(index='Symbols', columns='Attributes')

In [12]:
# 日付の降順に並べ替え
df_stock2.sort_values(by=['Date', 'Attributes'], ascending=[False, True], axis=1, inplace=True)

In [13]:
# マルチカラムをリセット
df_stock3 = df_stock2.T.reset_index(drop=False).T

In [14]:
# 既存のカラムを削除
df_stock4 = df_stock3.drop(df_stock3.index[[0,1]])

In [15]:
# 予想前日の株価データにINNR4の結果をマージ
df_result = pd.merge(df_stock4, df_idnr, left_index=True, right_index=True, how='left', indicator=True)

In [16]:
# 不要な列を削除 
df_result.drop(columns=['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'diff'], inplace=True)

In [17]:
df_result.replace({'both':1, 'left_only':0, '':0}, inplace=True)

In [18]:
# 列名を置換(IDNR4要素追加完了)
df_result.rename(columns = {'_merge':'INDR4'}, inplace=True)

In [19]:
# 先ほど作成した表にタートルスープ買いデータの結果をマージ
df_result1 = pd.merge(df_result, df_tsplusb, left_index=True, right_index=True, how='left', indicator=True)

In [20]:
# 不要な列を削除 
df_result1.drop(columns=['Adj Close', 'Low', 'Adjdate', 'Lowdate'], inplace=True)

In [21]:
df_result1.replace({'both':1, 'left_only':0, '':0}, inplace=True)

In [22]:
# 列名を置換(タートルスープ買いの要素追加完了)
df_result1.rename(columns ={'_merge':'TSPLB'} , inplace=True)

In [23]:
# 先ほど作成した表にタートルスープ売りデータの結果をマージ
df_result2 = pd.merge(df_result1, df_tspluss, left_index=True, right_index=True, how='left', indicator=True)

In [24]:
# 不要な列を削除 
df_result2.drop(columns=['Adj Close', 'High', 'Adjdate', 'Highdate'], inplace=True)

In [25]:
df_result2.replace({'both':1, 'left_only':0, '':0}, inplace=True)

In [26]:
# 列名を置換(タートルスープ売りの要素追加完了)
df_result2.rename(columns ={'_merge':'TSPLS'} , inplace=True)

In [27]:
# 先ほど作成した表にピンボール買いデータの結果をマージ
df_result3 = pd.merge(df_result2, df_pinb, left_index=True, right_index=True, how='left', indicator=True)

In [28]:
# 不要な列を削除 
df_result3.drop(columns=['Adj Close', 'Close', 'High', 'Low', 'Open', 'RSI', 'RSI_diff', 'Volume', 'diff', 'eighty', 'twenty'], inplace=True)

In [29]:
df_result3.replace({'both':1, 'left_only':0, '':0}, inplace=True)

In [30]:
# 列名を置換(ピンボール買いの要素追加完了)
df_result3.rename(columns ={'_merge':'PIN80B'} , inplace=True)

In [31]:
# 先ほど作成した表にピンボール売りデータの結果をマージ
df_result4 = pd.merge(df_result3, df_pins, left_index=True, right_index=True, how='left', indicator=True)

In [32]:
# 不要な列を削除 
df_result4.drop(columns=['Adj Close', 'Close', 'High', 'Low', 'Open', 'RSI', 'RSI_diff', 'Volume', 'diff', 'eighty', 'twenty'], inplace=True)

In [33]:
df_result4.replace({'both':1, 'left_only':0, '':0}, inplace=True)

In [34]:
# 列名を置換(ピンボール売りの要素追加完了)
df_result4.rename(columns ={'_merge':'PIN80S'} , inplace=True)

In [35]:
# その他SBIスクリーニングデータを追加
df_screen = pd.read_csv('data/screener_result_' + filedate + '.csv', index_col=0, header=[0], parse_dates=[0])

In [36]:
# 結合キー”株価コード＋".T"を作成
df_screen['key'] = df_screen.index.astype(str) + ".T"

In [37]:
# AIへのinputデータ元作成
df_testdate = pd.merge(df_screen, df_result4, left_on='key', right_index=True, how='inner', indicator=True)

In [38]:
# 不要な列を削除 
df_testdate1 = df_testdate.drop(columns=['銘柄名', '市場', '現在値', '前日比(%)',  '_merge'])

In [39]:
# 列名を置換(ピンボール売りの要素追加完了)
df_testdate1.rename(columns ={'PER(株価収益率)(倍)':'PER', 'PBR(株価純資産倍率)(倍)':'PBR', 
                              '配当利回り(%)':'HAITO', 'ROE(自己資本利益率)(%)':'ROE',
                              'ROA(総資産当期利益率)(%)':'ROA', '株価移動平均線乖離率(%)':'MA05',
                              '株価移動平均線乖離率(%).1':'MA25', '株価移動平均線乖離率(%).2':'MA75','RSI(%)':'RSI', 
                              0:'AdjClose_c',1:'Close_c',2:'High_c',3:'Low_c',4:'Open_c',5:'Volume_c',
                              6:'AdjClose_p',7:'Close_p',8:'High_p',9:'Low_p',10:'Open_p',11:'Volume_p',
                              } , inplace=True)

In [40]:
# PERの値に含まれている”,”を削除 regex=Trueをすることによって、対象の文字列のみ置き換わる
df_testdate2 = df_testdate1.replace({'PER': {',':''}}, regex=True)

In [41]:
# PERをobject属性から'float64'属性に変更
df_testdate2['PER'] = df_testdate2['PER'].astype('float64')

In [42]:
# 終値の前日比を計算して格納
df_testdate2['Diff'] = (df_testdate2['AdjClose_c'] - df_testdate2['AdjClose_p']) / df_testdate2['AdjClose_c'] 

In [43]:
# 企業の財務情報を追加（00_inputdata_impriment）
df_finance = pd.read_csv('data/finance_' + filedate + '.csv', index_col=0, header=[0], parse_dates=[0])

In [44]:
# AIへのinputデータ元作成
df_testdate3 = pd.merge(df_testdate2, df_finance, left_index=True, right_index=True)

In [45]:
pd.set_option('display.max_rows', 60) 
# defult:60,制限なし:None
pd.set_option('display.max_columns', None) 
# defult:20,制限なし:None
df_testdate3

,PER,PBR,HAITO,ROE_x,ROA_x,MA05,MA25,MA75,RSI,key,AdjClose_c,Close_c,High_c,Low_c,Open_c,Volume_c,AdjClose_p,Close_p,High_p,Low_p,Open_p,Volume_p,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,INDR4,TSPLB,TSPLS,PIN80B,PIN80S,Diff,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン,売上高,営業利益,経常利益,純利益,EPS,ROE_y,ROA_y,一株配当,剰余金の配当,自社株買い,配当性向,総還元性向,純資産配当率
コード,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1301,8.96,0.85,2.50,10.5,4.3,-1.11,-3.56,-1.28,34.88,1301.T,3200.0,3200.0,3260.0,3195.0,3250.0,19700.0,3265.0,3265.0,3290.0,3245.0,3265.0,16000.0,3220.0,3220.0,3230.0,3180.0,3180.0,16600.0,3245.0,3245.0,3245.0,3215.0,3230.0,12100.0,3250.0,3250.0,3255.0,3230.0,3230.0,13200.0,3225.0,3225.0,3230.0,3185.0,3190.0,16300.0,3185.0,3185.0,3210.0,3180.0,3205.0,14200.0,3205.0,3205.0,3230.0,3170.0,3230.0,14000.0,3235.0,3235.0,3250.0,3225.0,3250.0,9400.0,3260.0,3260.0,3265.0,3240.0,3260.0,8200.0,0,0,0,0,0,-0.020313,1.163310e+11,3.997500e+10,3.530000e+10,2.873700e+10,8.056000e+09,3.272000e+10,3753.75,34.7,5.997000e+09,-5.270000e+08,-4.627000e+09,0.000000e+00,7.097000e+09,2.41,2.491970e+11,4.657000e+09,4.879000e+09,3.838000e+09,356.90,9.50,3.30,80.0,7.540000e+08,82000000.0,19.65,21.78,2.4
1352,12.76,1.46,1.64,12.0,2.9,-0.02,-0.10,16.83,40.00,1352.T,1217.0,1217.0,1218.0,1217.0,1217.0,8300.0,1217.0,1217.0,1219.0,1217.0,1217.0,12000.0,1217.0,1217.0,1218.0,1217.0,1217.0,20800.0,1217.0,1217.0,1218.0,1217.0,1217.0,20900.0,1218.0,1218.0,1218.0,1217.0,1217.0,1000.0,1217.0,1217.0,1218.0,1217.0,1218.0,700.0,1218.0,1218.0,1218.0,1216.0,1216.0,2900.0,1218.0,1218.0,1218.0,1217.0,1217.0,900.0,1217.0,1217.0,1217.0,1217.0,1217.0,400.0,1217.0,1217.0,1218.0,1217.0,1217.0,1700.0,0,0,0,0,0,0.000000,4.138800e+10,7.855000e+09,6.946000e+09,3.243000e+09,1.900000e+09,1.878700e+10,834.05,16.9,4.367000e+09,-1.817000e+09,-6.620000e+08,-6.110000e+08,4.009000e+09,5.24,8.326500e+10,1.230000e+09,1.163000e+09,7.980000e+08,95.30,11.43,1.93,20.0,1.670000e+08,0.0,20.93,0.00,2.4
1376,14.06,0.93,1.68,6.8,3.7,3.07,6.28,10.81,68.28,1376.T,1731.0,1731.0,1753.0,1657.0,1684.0,39300.0,1684.0,1684.0,1702.0,1630.0,1700.0,28800.0,1700.0,1700.0,1705.0,1648.0,1664.0,23400.0,1629.0,1629.0,1657.0,1629.0,1653.0,8800.0,1653.0,1653.0,1668.0,1634.0,1641.0,20900.0,1640.0,1640.0,1650.0,1630.0,1646.0,12300.0,1636.0,1636.0,1646.0,1595.0,1600.0,14500.0,1582.0,1582.0,1582.0,1557.0,1581.0,11200.0,1581.0,1581.0,1610.0,1581.0,1610.0,13100.0,1603.0,1603.0,1608.0,1576.0,1576.0,13300.0,0,0,0,0,0,0.027152,4.745203e+10,2.169695e+10,2.083818e+10,1.771199e+10,0.000000e+00,0.000000e+00,1859.82,45.7,2.294651e+09,-8.389950e+08,-3.847630e+08,-1.028317e+09,5.486002e+09,3.78,6.077909e+10,1.661398e+09,1.765679e+09,1.436066e+09,123.09,6.62,3.03,29.0,3.286890e+08,781000.0,22.89,22.94,1.6
1377,24.96,1.71,0.89,7.2,7.9,0.09,8.49,19.41,79.72,1377.T,4275.0,4275.0,4345.0,4265.0,4265.0,255500.0,4290.0,4290.0,4330.0,4255.0,4315.0,233400.0,4325.0,4325.0,4385.0,4195.0,4250.0,321600.0,4225.0,4225.0,4310.0,4205.0,4260.0,181200.0,4240.0,4240.0,4270.0,4200.0,4200.0,221100.0,4165.0,4165.0,4210.0,4125.0,4155.0,138500.0,4095.0,4095.0,4115.0,4000.0,4020.0,120300.0,4000.0,4000.0,4005.0,3935.0,3965.0,120400.0,4015.0,4015.0,4085.0,3960.0,4050.0,233700.0,4065.0,4065.0,4120.0,4020.0,4085.0,240900.0,0,0,0,0,0,-0.003509,1.330770e+11,1.118980e+11,1.109050e+11,9.109900e+10,1.894000e+09,8.940000e+08,2503.80,83.9,1.136200e+10,-5.165000e+09,-4.005000e+09,-4.689000e+09,1.475800e+10,16.41,6.921800e+10,9.725000e+09,1.007800e+10,7.636000e+09,171.22,6.84,5.74,38.0,1.472000e+09,2000000.0,19.28,19.30,1.6
1379,15.50,1.16,3.00,7.7,6.5,-0.03,1.17,1.80,59.86,1379.T,1997.0,1997.0,2018.0,1994.0,2000.0,98400.0,2010.0,2010.0,2018.0,2001.0,2005.0,80100.0,1998.0,1998.0,2003.0,1966.0,1970.0,56200.0,1989.0,1989.0,2000.0,1984.0,1990.0,37700.0,1994.0,1994.0,2012.0,1994.0,1997.0,93400.0,1996.0,1996.0,

In [46]:
# 行にNaNデータがある銘柄をリスト化（後で再取得を計る）
df_nanlist = df_testdate3[df_testdate3.isnull().any(axis=1)]

In [47]:
# データチェック
df_testdate4 = df_testdate3.dropna(how='any')

In [48]:
# データチェック
df_testdate4.isnull().any()

PER       False
PBR       False
HAITO     False
ROE_x     False
ROA_x     False
          ...  
剰余金の配当    False
自社株買い     False
配当性向      False
総還元性向     False
純資産配当率    False
Length: 103, dtype: bool

In [49]:
df_testdate4.to_csv('data/inputAI_' + filedate + '.csv')

In [50]:
df_nanlist.to_csv('data/nanlist_' + filedate + '.csv')